In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def scrape_stilltasty(index):
    # URL of the page to scrape
    url = f'https://www.stilltasty.com/Fooditems/index/{index}'

    # Send a GET request to the URL
    response = requests.get(url)

    # Initialize a dictionary to store values
    shelf_life = {}

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find first storage location
        storage_location_elements = soup.find_all('div', class_='food-storage-left')

        for e in storage_location_elements:
            storage_location = e.text.strip()
            # Find associated shelf life information
            shelf_life_info_element = e.find_next_sibling('div')
            shelf_life_info = shelf_life_info_element.text.strip()
            shelf_life[storage_location] = shelf_life_info

        return shelf_life
    else:
        # Print an error message if the request was not successful
        print(f"Error: Failed to retrieve data from {url}. Status code: {response.status_code}")

In [13]:
def get_index_numbers(url,target_string,get_names=False):
    index_numbers = []
    object_names = []

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all links with href attribute containing target_string
        links = soup.find_all('a', href=lambda href: href and target_string in href)
        #Remove image links
        links = [link for link in links if not link.find('img')]
        # Extract index numbers from href attributes
        for link in links:
            href = link['href']
            index_number = href.split('/')[-1]
            index_numbers.append(index_number)
            if get_names:
                object_name = link.text.strip()
                object_names.append(object_name)
            
        return index_numbers, object_names 
    
    else:
        # Print an error message if the request was not successful
        print(f"Error: Failed to retrieve data from {url}. Status code: {response.status_code}")

In [16]:
# URL of the StillTasty index page
url = 'https://www.stilltasty.com/Fooditems/index'
target_string = "/searchitems/index/"
category_index_numbers, categories = get_index_numbers(url,target_string,True)
print(category_index_numbers)
print(categories)

['26', '25', '9', '27', '7', '28', '6', '31', '30', '5']
['Fruits', 'Vegetables', 'Dairy & Eggs', 'Meat & Poultry', 'Fish & Shellfish', 'Nuts, Grains & Pasta', 'Condiments & Oils', 'Snacks & Baked Goods', 'Herbs & Spices', 'Beverages']


In [15]:
print(len(category_index_numbers))
print(len(categories))

10
10


In [ ]:
index_len = 0
all_index_numbers = []
all_categories = []
for x,i in enumerate(category_index_numbers):
    print(f"Now scraping index {i}")
    url = f'https://www.stilltasty.com/searchitems/index/{i}'
    index_numbers, item_names = get_index_numbers(url,"/Fooditems/index/")
    if index_numbers:
        index_len += len(index_numbers)
        print(index_len)
        all_index_numbers = all_index_numbers + index_numbers
        all_categories = all_categories + [categories[x]]*len(index_numbers)
    print(f'Completed index {i}')
    sleep(3)
    other_pages, page_names = get_index_numbers(url,"/searchitems/index/")
    print(other_pages)
    if other_pages:
        for j in other_pages: 
            print(f"Now scraping index {j}")
            url = f'https://www.stilltasty.com/searchitems/index/{i}'
            index_numbers, item_names = get_index_numbers(url,"/Fooditems/index/")
            if index_numbers:
                index_len += len(index_numbers)
                print(index_len)
                all_index_numbers = all_index_numbers + index_numbers
                all_categories = all_categories + [categories[x]]*len(index_numbers)
            print(f'Completed index {j}')
            sleep(3)
            
print(all_index_numbers)
print(all_categories)

In [ ]:
index_len

In [ ]:
url = f'https://www.stilltasty.com/searchitems/index/{i}'
target_string = '/Fooditems/index/'
get_index_numbers(url,target_string)

In [ ]:
index_numbers = []
object_names = []

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all links with href attribute containing target_string
    links = soup.find_all('a', href=lambda href: href and target_string in href)
    # links = [x for i,x in enumerate(links) if i%2 != 0]
    # Extract index numbers from href attributes
    for link in links:
        href = link['href']
        index_number = href.split('/')[-1]
        index_numbers.append(index_number)


In [ ]:
links

In [ ]:
for i in all_index_numbers:
    # scrape_stilltasty(i)

In [ ]:
len(all_index_numbers)

In [ ]:
from collections import Counter
Counter(all_categories)

In [ ]:
scrape_stilltasty('17634')